In [6]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# corpus_root = 'drive/My Drive/Colab Notebooks/data science and machine learning/Boy or Girl/'
# raw_data = pd.read_csv(corpus_root+'boy or girl 2024 train_missingValue.csv')

raw_data = pd.read_csv("boy or girl 2024 train_missingValue.csv")
raw_data

,id,gender,star_sign,phone_os,height,weight,sleepiness,iq,fb_friends,yt,self_intro
0,1,2,處女座,Apple,154.0,43.0,2.0,180.0,583.0,0,Beautiful
1,2,2,處女座,NaN,156.0,47.0,2.0,130.0,400.0,3.5,Enjoying being who I'm notsss
2,3,1,射手座,Android,170.0,61.0,3.0,90.0,540.0,5,Practice Makes perfect
3,4,1,射手座,Apple,170.0,62.0,4.0,100.0,173.0,5,Straightforward
4,5,2,射手座,Android,158.0,NaN,3.0,NaN,NaN,1.2,Humorous
...,...,...,...,...,...,...,...,...,...,...,...
418,419,1,處女座,Android,166.0,66.0,4.0,90.0,1000.0,1,I hope i am a super hero.
419,420,1,牡羊座,Android,176.0,65.0,4.0,87.0,1300.0,2,God damn dope
420,421,1,天秤座,Apple,174.0,72.0,NaN,NaN,NaN,NaN,Sunny
421,422,2,天蠍座,Apple,167.0,50.0,3.0,180.0,483.0,10,Light


In [2]:
self_intro_data = pd.read_csv("gender_self_intro.csv")
self_intro_data

,id,gender,self_intro
0,1,2,Beautiful
1,2,2,Enjoying being who I'm notsss
2,3,1,Practice Makes perfect
3,4,1,Straightforward
4,5,2,Humorous
...,...,...,...
418,419,1,I hope i am a super hero.
419,420,1,God damn dope
420,421,1,Sunny
421,422,2,Light


In [3]:
# test_data = pd.read_csv(corpus_root+'boy or girl 2024 test no ans_missingValue.csv')
test_data = pd.read_csv("boy or girl 2024 test no ans_missingValue.csv")
test_data

,id,gender,star_sign,phone_os,height,weight,sleepiness,iq,fb_friends,yt,self_intro
0,1,0,天蠍座,Apple,NaN,100.0,1.0,87.000,87.0,87.0,GOod
1,2,0,金牛座,Apple,175.0,80.0,3.0,130.000,2000.0,30.0,Easygoing
2,3,0,雙子座,Apple,155.0,45.0,3.0,150.000,400.0,9.0,I LOVE INTEL
3,4,0,處女座,Apple,173.0,85.0,4.0,100.000,2000.0,15.0,"I'm a hard-work man, just do my best to finish..."
4,5,0,射手座,Android,164.0,57.0,4.0,130.000,505.0,2.0,I'm smart
...,...,...,...,...,...,...,...,...,...,...,...
390,391,0,處女座,Android,160.0,48.0,3.0,75.000,98.0,2.0,Starting by Starting
391,392,0,NaN,Apple,170.0,NaN,NaN,105.000,510.0,NaN,A little bit smart
392,393,0,金牛座,Apple,160.0,45.0,4.0,100.000,600.0,2000.0,Hi
393,394,0,巨蟹座,NaN,180.0,NaN,NaN,199.999,NaN,60.0,"I'm not beautiful, but smart 😀"


In [4]:
import datetime
import pytz
def export_csv(df):
  now = datetime.datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
  formatted_time = now.strftime('%Y-%m-%d %H時%M分%S秒')
  df.to_csv(formatted_time + "predict.csv", index=False,encoding="utf_8_sig")
  # df.to_csv(corpus_root + formatted_time + "predict.csv", index=False,encoding="utf_8_sig")

### 前處理 刪掉星座和手機

In [9]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


# 读取数据
data = raw_data

# 文本预处理函数
def preprocess_text(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除标点符号和停用词
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    return tokens

# 对文本进行预处理
data['tokens'] = data['self_intro'].apply(preprocess_text)

# 训练 Word2Vec 模型
model = Word2Vec(data['tokens'], vector_size=100, window=5, min_count=1, sg=0)

# 创建 Word2Vec 特征函数
def word2vec_feature(tokens):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# 应用 Word2Vec 特征函数
data['word2vec_feature'] = data['tokens'].apply(word2vec_feature)

# 将特征添加到原始数据中
data = pd.concat([data.drop(columns=['tokens']), pd.DataFrame(data['word2vec_feature'].tolist())], axis=1)

# 将数据集分为性别为1和性别为2的两个子数据集
gender1_data = data[data['gender'] == 1]
gender2_data = data[data['gender'] == 2]

# 可以进一步使用 gender1_data 和 gender2_data 来训练分类器或进行其他分析任务

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 

# 讀取訓練資料集
train_data = pd.read_csv(r"..\KNN\dataset\KNN_without_outlier.csv")

# 將self_intro欄位從訓練資料中移除，因為這裡不打算使用該欄位作為特徵
train_data = train_data.drop(columns=['self_intro'])

# 將性別標籤設置為0和1，其中1代表男性，2代表女性
train_data['gender'] = train_data['gender'].apply(lambda x: 1 if x == 1 else 0)

# 將資料集分為特徵（X）和標籤（y）
X = train_data.drop(columns=['gender'])
y = train_data['gender']

# 將資料集分為訓練集和驗證集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化標準化器
scaler = StandardScaler()

# 對訓練集和驗證集進行標準化
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 初始化隨機森林分類器
rf = RandomForestClassifier(random_state=42)

# 定義超參數範圍
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 使用 RapidSearch (GridSearchCV) 進行超參數優化
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# 獲取最佳模型
best_rf = grid_search.best_estimator_

# 在驗證集上進行預測
val_predictions = best_rf.predict(X_val_scaled)

# 計算模型在驗證集上的準確率
accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", accuracy)

# 進行測試資料集的預測
test_data = pd.read_csv(r"..\KNN\dataset\test_KNN_without_outlier.csv")
test_data = test_data.drop(columns=['self_intro', 'id', 'gender'])
test_data_scaled = scaler.transform(test_data)
test_predictions = best_rf.predict(test_data_scaled)

# 建立新的 DataFrame 來存放預測結果
result_df = pd.DataFrame({'ID': range(1, len(test_predictions) + 1), 'gender': [2 if pred == 0 else pred for pred in test_predictions]})

# 將結果存入新的 CSV 檔案中
result_df.to_csv('prediction_result_RF_rapid.csv', index=False)

# 輸出預測結果
print(result_df)


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 

# 读取数据
data = raw_data

# 文本预处理函数
def preprocess_text(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除标点符号和停用词
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    return tokens

# 对文本进行预处理
data['tokens'] = data['self_intro'].apply(preprocess_text)

# 训练 Word2Vec 模型
model = Word2Vec(data['tokens'], vector_size=100, window=5, min_count=1, sg=0)

# 创建 Word2Vec 特征函数
def word2vec_feature(tokens):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# 应用 Word2Vec 特征函数
data['word2vec_feature'] = data['tokens'].apply(word2vec_feature)

# 将特征添加到原始数据中
data = pd.concat([data.drop(columns=['tokens']), pd.DataFrame(data['word2vec_feature'].tolist())], axis=1)

# 将数据集分为性别为1和性别为2的两个子数据集
gender1_data = data[data['gender'] == 1]
gender2_data = data[data['gender'] == 2]

# 可以进一步使用 gender1_data 和 gender2_data 来训练分类器或进行其他分析任务

# 将self_intro欄位從訓練資料中移除，因為這裡不打算使用該欄位作為特徵
train_data = pd.read_csv(r"..\KNN\dataset\KNN_without_outlier.csv")
train_data = train_data.drop(columns=['self_intro'])

# 将性别标签设置為0和1，其中1代表男性，2代表女性
train_data['gender'] = train_data['gender'].apply(lambda x: 1 if x == 1 else 0)

# 将資料集分为特徵（X）和标签（y）
X = train_data.drop(columns=['gender'])
y = train_data['gender']

# 将資料集分为訓練集和驗證集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化标准化器
scaler = StandardScaler()

# 对訓練集和驗證集進行標準化
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 初始化随机森林分類器
rf = RandomForestClassifier(random_state=42)

# 定义超参数范围
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 使用 RapidSearch (GridSearchCV) 進行超参数优化
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# 获取最佳模型
best_rf = grid_search.best_estimator_

# 在驗證集上進行預測
val_predictions = best_rf.predict(X_val_scaled)

# 計算模型在驗證集上的準確率
accuracy = accuracy_score(y_val, val_predictions)
print("Validation Accuracy:", accuracy)

# 进行测试資料集的预测
test_data = pd.read_csv(r"..\KNN\dataset\test_KNN_without_outlier.csv")
test_data = test_data.drop(columns=['self_intro', 'id', 'gender'])
test_data_scaled = scaler.transform(test_data)
test_predictions = best_rf.predict(test_data_scaled)

# 建立新的 DataFrame 來存放預測結果
result_df = pd.DataFrame({'ID': range(1, len(test_predictions) + 1), 'gender': [2 if pred == 0 else

In [14]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 
import numpy as np

train_data = pd.read_csv(r"..\KNN\dataset\KNN_without_outlier.csv")
train_text_data = train_data.copy()

# 文本预处理函数
def preprocess_text(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除标点符号和停用词
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    return tokens

# 对文本进行预处理
train_text_data['tokens'] = train_text_data['self_intro'].apply(preprocess_text)

# 训练 Word2Vec 模型
model = Word2Vec(train_text_data['tokens'], vector_size=100, window=5, min_count=1, sg=0)

# 创建 Word2Vec 特征函数
def word2vec_feature(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# 应用 Word2Vec 特征函数
train_text_data['word2vec_feature'] = train_text_data['tokens'].apply(word2vec_feature)


# 从原始数据中移除'tokens'和'self_intro'列
X_text_features = pd.DataFrame(train_text_data['word2vec_feature'].tolist())

# 从原始数据中移除'tokens'和'self_intro'列
train_data = train_data.drop(columns=['self_intro','gender'])

# 标准化非文本特征
scaler = StandardScaler()
X_non_text_features = scaler.fit_transform(train_data)

# 将文本特征和非文本特征结合
X = np.hstack((X_text_features, X_non_text_features))

array([[ 8.13288987e-03, -4.45661834e-03, -1.06700230e-03, ...,
         1.50306921e+00, -5.85793799e-02, -1.38824952e-01],
       [-2.37336010e-03,  3.95035231e-03,  3.54133267e-03, ...,
         9.45725371e-02, -5.89446858e-02, -1.34398690e-01],
       [-2.25528912e-03,  3.27215041e-03, -4.91253508e-04, ...,
        -1.03222480e+00, -5.86652168e-02, -1.32501721e-01],
       ...,
       [ 5.55166230e-03, -3.48776113e-04, -9.96322487e-04, ...,
        -1.11673460e+00, -5.71480992e-02, -1.36295659e-01],
       [ 2.09900143e-04,  3.22359335e-03, -1.63235550e-03, ...,
         1.50306921e+00, -5.87790006e-02, -1.26178490e-01],
       [ 4.79276711e-03, -3.63464421e-03, -4.25703824e-03, ...,
        -1.70830320e+00, -5.91443066e-02, -1.37231498e-01]])

In [15]:
import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 

train_data = pd.read_csv(r"..\KNN\dataset\KNN_without_outlier.csv")
train_text_data = train_data.copy()

# 文本预处理函数
def preprocess_text(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除标点符号和停用词
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords.words('english')]
    return tokens

# 对文本进行预处理
train_text_data['tokens'] = train_text_data['self_intro'].apply(preprocess_text)

# 训练 Word2Vec 模型
model = Word2Vec(train_text_data['tokens'], vector_size=100, window=5, min_count=1, sg=0)

# 创建 Word2Vec 特征函数
def word2vec_feature(tokens):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

# 应用 Word2Vec 特征函数
train_text_data['word2vec_feature'] = train_text_data['tokens'].apply(word2vec_feature)


# 从原始数据中移除'tokens'和'self_intro'列
X_text_features = pd.DataFrame(train_text_data['word2vec_feature'].tolist())

y = train_data['gender']

# 从原始数据中移除'tokens'和'self_intro'列
train_data = train_data.drop(columns=['self_intro','gender'])

# 标准化非文本特征
scaler = StandardScaler()
X_non_text_features = scaler.fit_transform(train_data)

# 将文本特征和非文本特征结合
X = np.hstack((X_text_features, X_non_text_features))

# 分割数据集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化和训练随机森林分类器
# 您可以根据需要调整这些参数
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 在验证集上预测并计算准确率
y_pred = rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.8674698795180723
